# Scientific Computing: Assignment Set 1
_Boyan Mihaylov (15067602)_

## 0. Imports / Preliminaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from importlib import reload

Two external Python modules are loaded: one for the DLA model and one for the Gray-Scott reaction-diffusion model.

In [2]:
import dla as dla
import gray_scott as gs

## 1. DLA Based on the Time-Independent Diffusion Equation

In [133]:
reload(dla)

<module 'dla' from 'd:\\GitHub\\scented-candles\\Assignment Set 2\\dla.py'>

In [134]:
# Simulation parameters
size = 10

dla.run_dla_diff_equation(size, 1, 1.95, 1.0)

[[5 0]]
[5 1]
[[5 0]
 [5 1]]
